In [15]:
from asf_core_data import get_mcs_installations, load_preprocessed_epc_data

import pandas as pd
import numpy as np

In [87]:
# change path to your own local version of EPC data
epc_path = "/Users/chris.williamson/Documents/ASF_data"

In [25]:
mcs = get_mcs_installations("full")

Loading /outputs/MCS/mcs_installations_epc_full_230510.csv from S3


In [26]:
# convert date columns to datetime type
mcs["commission_date"] = pd.to_datetime(mcs["commission_date"])
mcs["INSPECTION_DATE"] = pd.to_datetime(mcs["INSPECTION_DATE"])

In [32]:
# merge installation type columns and filter to domestic installations
mcs["installation_type"] = mcs["installation_type"].fillna(
    mcs["end_user_installation_type"]
)
mcs = mcs.loc[mcs.installation_type == "Domestic"].reset_index(
    drop=True
)

### What proportion of records in the MCS database relate to new-build installations?

In [33]:
# filter to first records - if property is a new build then it should appear as such in its first EPC record
first_records = (
    mcs
    .sort_values("INSPECTION_DATE")
    .groupby("original_mcs_index")
    .head(1)
    .sort_values("original_mcs_index")
)

In [34]:
# find number of days between first recorded EPC inspection and HP commission
first_records["diff_epc_to_mcs"] = (
    first_records["commission_date"] - first_records["INSPECTION_DATE"]
).dt.days

# assume dwelling was built with HP if:
# - first EPC shows it as a new dwelling
# - time difference between EPC inspection when dwelling was built and HP installation is less than 1 year
first_records["assumed_hp_when_built"] = (
    first_records["TRANSACTION_TYPE"] == "new dwelling"
) & (first_records["diff_epc_to_mcs"] < 365)

Proportions of new build installations in the MCS database:

In [38]:
first_records.assumed_hp_when_built.value_counts(normalize=True)

False    0.881317
True     0.118683
Name: assumed_hp_when_built, dtype: float64

Top 5 installers of new build installations:

In [85]:
first_records.loc[first_records["assumed_hp_when_built"]]["installer_name"].value_counts().head()

Nu-Heat UK Ltd                                                        1039
Alto Energy Ltd                                                        371
Grant Engineering (UK) Ltd                                             340
Total Renewable Solutions SW Limited T/A Total Renewable Solutions     306
Kensa Heat Pumps Ltd                                                   205
Name: installer_name, dtype: int64

Difference in average costs for retrofits and new builds:

In [42]:
first_records.groupby("assumed_hp_when_built").cost.mean()

assumed_hp_when_built
False    12619.088613
True     11601.417169
Name: cost, dtype: float64

### What proportion of properties in the EPC database that were built with a HP appear in the MCS database?

In [88]:
epc = load_preprocessed_epc_data(epc_path, version="preprocessed", usecols=["UPRN", "TRANSACTION_TYPE", "HP_INSTALLED"])

In [77]:
# filter to records of new builds with a heat pump
new_hp = epc.loc[(epc["TRANSACTION_TYPE"] == "new dwelling") & (epc["HP_INSTALLED"])]

In [79]:
# replace missing or unknown UPRNs to avoid them appearing the same in both datasets
new_hp["UPRN"] = new_hp["UPRN"].replace("unknown", np.nan).fillna(0).astype("float").astype("int")
mcs["UPRN"] = mcs["UPRN"].replace("unknown", np.nan).fillna(-1).astype("float").astype("int")

/var/folders/82/3twghjpj20j97kv1v40lw6j80000gn/T/ipykernel_32201/1882580727.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_hp["UPRN"] = new_hp["UPRN"].replace("unknown", np.nan).fillna(0).astype("float").astype("int")


In [80]:
new_hp["in_mcs"] = new_hp["UPRN"].isin(mcs["UPRN"])

/var/folders/82/3twghjpj20j97kv1v40lw6j80000gn/T/ipykernel_32201/888640001.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_hp["in_mcs"] = new_hp["UPRN"].isin(mcs["UPRN"])


Proportions of EPC new builds that are in the MCS database:

In [89]:
new_hp["in_mcs"].value_counts(normalize=True)

False    0.800795
True     0.199205
Name: in_mcs, dtype: float64